In [2]:
#########################################
############# SEARCH & LLM  #############
#########################################

import openai 
import os
from dotenv import load_dotenv
# import numpy as np
load_dotenv()
#
openai.api_type = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

from pymilvus import connections, Collection 

In [3]:
COLLECTION_NAME = os.getenv("COLLECTION_NAME")
EMBEDDING_MODEL = os.getenv("OPENAI_ENGINE")
MILVUS_HOST = "23.236.50.189"
MILVUS_PORT = "19530"
DIMENSION = 1536 
connections.connect(host=MILVUS_HOST, port=MILVUS_PORT) 
 
collection = Collection(name=COLLECTION_NAME)

In [4]:

# Vektörleştirme modelini
from openai import AzureOpenAI
 
client = AzureOpenAI( 
    api_version=openai.api_version , 
    api_key= openai.api_key,
    azure_endpoint=openai.api_base,
)

def generate_embeddings(texts):
    return [client.embeddings.create(input=[text], model="text-embedding-ada-002").data[0].embedding for text in texts]


In [5]:
QUERY_PARAM = {
    "metric_type": "L2",
    "params": {"ef": 64},   
}
def search_milvus(queries):
    top_k = 5
    if type(queries) != list:
        queries = [queries]
    documents = collection.search(generate_embeddings(queries), 
                            anns_field='embedding', param=QUERY_PARAM, limit=top_k,
                            output_fields=['page_content', 'source'])
    context=""
    for values in documents:
        for doc in values:
            context += doc.entity._row_data["page_content"] + " link: " + doc.entity._row_data["source"] + " "
    
        #collection.release()
    print(f"context:"{context}) 
    return context
  

In [6]:
# AzureOpenAI LLM ile cevap üretme fonksiyonu
def generate_answer(question, context): 
    prompt = f"""Given this text extracts:
    -----
    {context}
    -----
    Please answer with to the following question:
    Question: {question}
    Answer: """
    response = client.chat.completions.create(
        model="gpt-35-turbo",
        messages=[{"role": "user", "content": prompt  }]
      
    )

    #response.model_dump_json(indent=2)
    return response

In [9]:
# Kullanıcı girdisini alın ve işleyin
question = "Custom Measures metriclerini silmek için ne yapmalıyım ?"
retrieved_documents = search_milvus(question)    
answer = generate_answer(question, retrieved_documents)
print(answer.choices[0].message.content)

SonarQube emekli işlemleri. Custom Measures olarak adlandırılan gruplama ve hesap etiketlerinde paketin ilgili metriği Pasif olarak işaretlenmeli. (İlgili adres: https://sonar.softtech/custom_measures?id=ACIXXXXX) Bu işlem ön yüzden yapılabileceği gibi aşağıdaki adımları izleyerek API'ler aracılığı ile de yapılabilir. Mevcut Custom Measures metriclerini silmek için → Post api/custom_measures/delete Paketin emekli olduğuna dair ilgili metriği eklemek için → Post api/custom_measures/create Emekli link: https://wiki.softtech.com.tr/display/SDO/Get+Retired DevOps Künyesi Tüm işlerimizde her zaman göz önünde bulundurmamız gerekenler; İç müşterilerimizin memnuniyetini artırmalıyız . Yaptıklarımızla fark yaratmalıyız . Takım oyununu ve birlikteliği hep canlı tutmalıyız. Yaptığımız işlerdeki kaliteyi artırmalıyız . Sistematik çalışarak işlerimizi daha iyi yönetmeliyiz. Verimliliğimizi artırmalı ve kendi gelişimlerimizi de ihmal etmemeliyiz. The team Sinan Çayır About us Softtech DevOps Bilgi P